In [63]:
from helper import *
import sys

In [64]:
with open("./credentials.json") as r:
    config = json.loads(r.read())
minio_config = dict(
    key=config["accessKey"],
    secret=config["secretKey"],
    client_kwargs={
        'endpoint_url': config["url"]
    }
)

with open("./api_config.json") as r:
    config = json.loads(r.read())
api_config = dict(
    url=config["url"],
    token=config["token"],
)


In [65]:
url = api_config["url"]
token = api_config["token"]
headers = {"Authorization": "Bearer " + token}
topic = "service_ipd"
api_config["headers"] = headers
api_config["topic"] = topic

try:
  r = get_process(**api_config)
except Exception as ex:
  print(ex)
  sys.exit(0)

## set config to processing 
hospcode = r["hospcode"]
process_id = r["process_id"]
api_config["process_id"] = process_id
source_path = "s3://mophgw/{}".format(r["path"])
destination_path = "s3://phr/hisgw/{}/{}/{}".format(hospcode, topic,process_id)
local_code_path = "s3://mapcode/{}/localcode".format(hospcode)
std_code_path = "s3://mapcode/{}/stdcode".format(hospcode)

In [66]:
person = read_csv(minio_config, path=source_path, filename="person.csv.gz")

In [67]:
person_merge_local = dict(
    title=["title_code", "code"],
    gender=["gender", "code"],
    rh_blood_group=["rh_blood_group", "code"],
    blood_group=["blood_group", "code"],
    nationality=["nationality_code", "code"],
    marital_status=["marital_status", "code"],
)
person = merge_codes(minio_config, person, local_code_path,
                     prefix_type="local", merge_config=person_merge_local)

In [68]:
person_merge_std = dict(
    titles=["title_localcode", "localcode"],
    genders=["gender_localcode", "localcode"],
    rh_blood_groups=["rh_blood_group_localcode", "localcode"],
    blood_groups=["blood_group_localcode", "localcode"],
    nationalities=["nationality_localcode", "localcode"],
    marital_statuses=["marital_status_localcode", "localcode"],
)
person = merge_codes(minio_config, person, std_code_path,
                     prefix_type="std", merge_config=person_merge_std)

In [69]:
cols_rename = dict(
    group="groups",
    nationality="nationalities",
    status="statuses",
    gender="genders",
    title="titles"
)
person = rename_columns(person, cols_rename)

In [70]:
service_ipd_temp = read_csv(
    minio_config, path=source_path, filename="service_ipd.csv.gz")

# display(service_ipd)

In [71]:
service_ipd_temp = pd.merge(
    service_ipd_temp, person, how="left", left_on="hn", right_on="hn", suffixes=("", "_right"))
service_ipd_temp = service_ipd_temp[[
    a for a in service_ipd_temp.columns.to_list() if a.endswith("_right") == False]]

# display(service_ipd)

In [72]:
is_success = dict(
  person_telephone = False,
  person = False,
  person_death = False,
  service_ipd = False
)

In [73]:
person_telephone = dict()
person_telephone["cid"] = service_ipd_temp["cid"]
person_telephone["telephone"] = ""
person_telephone["type"] = ""
person_telephone["ext"] = ""

person_telephone = pd.DataFrame(person_telephone)
is_success["person_telephone"] = upload_minio(
    minio_config, person_telephone, f"{destination_path}/person_telephone.csv.gz", **api_config)

In [74]:
person = dict()
person["cid"] = service_ipd_temp["cid"]
person["birthdate"] = service_ipd_temp["birthdate"]
person["gender_code"] = service_ipd_temp["gender_stdcode"]
person["gender_name"] = service_ipd_temp["gender_stdname_th"]
person["rh_blood_group_code"] = service_ipd_temp["rh_blood_group_stdcode"]
person["rh_blood_group_name"] = service_ipd_temp["rh_blood_group_stdname_th"]
person["blood_group_code"] = service_ipd_temp["blood_group_stdcode"]
person["blood_group_name"] = service_ipd_temp["blood_group_stdname_th"]
person["title_code"] = service_ipd_temp["title_stdcode"]
person["title_name"] = service_ipd_temp["title_stdname_th"]
person["first_name"] = service_ipd_temp["first_name"]
person["middle_name"] = service_ipd_temp["middle_name"]
person["last_name"] = service_ipd_temp["last_name"]
person["nationality_code"] = service_ipd_temp["nationality_stdcode"]
person["nationality_name"] = service_ipd_temp["nationality_stdname_th"]
person["marital_status_code"] = service_ipd_temp["marital_status_stdcode"]
person["marital_status_name"] = service_ipd_temp["marital_status_stdname_th"]
person = pd.DataFrame(person)

is_success["person"] = upload_minio(
    minio_config, person, f"{destination_path}/person.csv.gz", **api_config)

In [75]:
person_death = dict()
person_death["cid"] = service_ipd_temp["cid"]
person_death["date_of_death"] = ""
person_death["cause_of_death"] = ""
person_death["place_of_death"] = ""
person_death = pd.DataFrame(person_death)

is_success["person_death"] = upload_minio(
    minio_config, person_death, f"{destination_path}/person_death.csv.gz", **api_config)

In [76]:
display(service_ipd_temp)

,hn,vn,an,visit_date,visit_time,admit_date,admit_time,discharge_date,discharge_time,discharge_reason,dchtype,cid,title_code,first_name,middle_name,last_name,birthdate,gender,rh_blood_group,blood_group,nationality_code,marital_status,title_localcode,title_localcode,title_localname,gender_localcode,gender_localcode,gender_localname,rh_blood_group_localcode,rh_blood_group_localcode,rh_blood_group_localname,blood_group_localcode,blood_group_localcode,blood_group_localname,nationality_localcode,nationality_localcode,nationality_localname,marital_status_localcode,marital_status_localcode,marital_status_localname,title_localcode,title_localcode,title_stdcode,title_stdname_th,title_stdname_en,gender_localcode,gender_localcode,gender_stdcode,gender_stdname_th,gender_stdname_en,rh_blood_group_localcode,rh_blood_group_localcode,rh_blood_group_stdcode,rh_blood_group_stdname_th,rh_blood_group_stdname_en,blood_group_localcode,blood_group_localcode,blood_group_stdcode,blood_group_stdname_th,blood_group_stdname_en,nationality_localcode,nationality_localcode,nationality_stdcode,nationality_stdname_th,nationality_stdname_en,marital_status_localcode,marital_status_localcode,marital_status_stdcode,marital_status_stdname_th,marital_status_stdname_en
0,000275351,650729193526,650729193526,2022-07-28T17:00:00.000Z,19:35:26,2022-07-28T17:00:00.000Z,20:33:00,2022-11-24T17:00:00.000Z,00:30:00,By Transfer,04,1319800154341,น.ส.,รัชดาภรณ์,<NA>,ปานกลาง,1995-11-18T17:00:00.000Z,2,<NA>,ไม่ทราบหมู่เลือด,99,1,น.ส.,<NA>,น.ส.,2,2,หญิง,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,ไทย,<NA>,<NA>,<NA>,น.ส.,<NA>,<NA>,<NA>,<NA>,2,2,2,หญิง,Female,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,99,ไทย,THAILAND,<NA>,<NA>,9,อื่นๆ,Other
1,000344731,650716093931,650716093931,2022-07-15T17:00:00.000Z,09:39:31,2022-07-15T17:00:00.000Z,10:33:00,2022-11-24T17:00:00.000Z,03:00:00,Dead Non Autopsy,09,3311200160438,นาย,ก้องไพร,<NA>,นุชผักแว่น,1963-06-05T17:00:00.000Z,1,<NA>,ไม่ทราบหมู่เลือด,99,1,นาย,<NA>,นาย,1,1,ชาย,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,ไทย,<NA>,<NA>,<NA>,นาย,<NA>,<NA>,<NA>,<NA>,1,1,1,ชาย,Male,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,99,99,99,ไทย,THAILAND,<NA>,<NA>,9,อื่นๆ,Other


In [77]:

service_ipd = dict()
service_ipd["cid"] = service_ipd_temp["cid"]
service_ipd["hospcode"] = hospcode
service_ipd["vn"] = service_ipd_temp["vn"]
service_ipd["an"] = service_ipd_temp["an"]
service_ipd["admit_date"] = service_ipd_temp["admit_date"]
service_ipd["admit_time"] = service_ipd_temp["admit_time"]
service_ipd["discharge_date"] = service_ipd_temp["discharge_date"]
service_ipd["discharge_time"] = service_ipd_temp["discharge_time"]
service_ipd["discharge_reason"] = service_ipd_temp["discharge_reason"]
service_ipd["discharge_type"] = service_ipd_temp["dchtype"]
service_ipd = pd.DataFrame(service_ipd_temp)
is_success["service_ipd"] = upload_minio(
    minio_config, service_ipd, f"{destination_path}/service_ipd.csv.gz", **api_config)

# display(service_ipd.__len__())    
# display(service_ipd)

{'person_telephone': True,
 'person': True,
 'person_death': True,
 'service_ipd': True}

In [78]:
if list(is_success.values()).count(False)>0:
  print("ERROR")
else:
  print("SUCCESS")

# display(is_success.__len__())
display(is_success)

SUCCESS


{'person_telephone': True,
 'person': True,
 'person_death': True,
 'service_ipd': True}